#  **Creating your own Dataset from Google Images**
   Inspired By Fastai  Library

### In this tutorial we will see how to easily create an image dataset through Google Images and load them in keras

Note The code has been taken from fastai library and tailored for keras



## Get a list of URLs
## Search and scroll

Go to [Google Images](https://images.google.com/) and search for the images you are interested in. The more specific you are in your Google Search, the better the results and the less manual pruning you will have to do.

Scroll down until you see a button that says **Show more results** . All the images you scrolled past are now available to download. To get more, click on the button. Then continue scrolling until you cannot scroll anymore. The maximum number of images Google Images shows is 700.

## Download into file

Now you must run some Javascript code in your browser which will save the URLs of all the images you want for you dataset.

Press Ctrl+Shift+J in Windows/Linux and Cmd+Opt+J in Mac for Chrome and Ctrl+Shift+K in Firefox, a small window the javascript 'Console' will appear. That is where you will paste the JavaScript commands.

You will need to get the urls of each of the images. You can do this by running the following commands:



```javascript
urls=Array.from(document.querySelectorAll('.rg_di.rg_meta')).map(el=>JSON.parse(el.textContent).ou);

window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```




## In this tutorial, we are working on birds datasets.

Here we have images of 10 different birds species found in India.
So the steps are as following
1. Make 10 CSV files of all the different birds in your local system by following above steps
2. Zip it and upload it to Google colab
3. Unzip it, download all the Images in their specific folder
4. Verify all the Images 
5. Use It and Enjoy

In [0]:
# import all the library's
from tqdm import tqdm # for progress bar 
import os,sys,requests 
from concurrent.futures import ProcessPoolExecutor ,as_completed # Asynchronous Execution
import numpy as np 
import matplotlib.pyplot as plt
from pathlib import Path
import keras
import tensorflow as tf


In [6]:
# Upload the ziped file 
from google.colab import files
uploaded = files.upload()

Saving Birds.zip to Birds.zip


In [7]:
!unzip Birds.zip
!ls -l Birds

Archive:  Birds.zip
   creating: Birds/
  inflating: Birds/Asian Paradise Flycatcher.csv  
  inflating: Birds/Himalayan Monal.csv  
  inflating: Birds/Indian Pitta.csv  
  inflating: Birds/Mrs. Gould’s Sunbird.csv  
  inflating: Birds/Oriental Dwarf Kingfisher.csv  
  inflating: Birds/Red Headed Trogon.csv  
  inflating: Birds/Sarus Crane.csv   
  inflating: Birds/Satyr Tragopan.csv  
  inflating: Birds/parrot.csv        
  inflating: Birds/peacock.csv       
total 620
-rw-rw-r-- 1 root root 61740 Nov  3 19:23 'Asian Paradise Flycatcher.csv'
-rw-rw-r-- 1 root root 48812 Nov  3 19:20 'Himalayan Monal.csv'
-rw-rw-r-- 1 root root 60080 Nov  3 19:15 'Indian Pitta.csv'
-rw-rw-r-- 1 root root 54367 Nov  3 19:12 'Mrs. Gould’s Sunbird.csv'
-rw-rw-r-- 1 root root 59841 Nov  3 19:06 'Oriental Dwarf Kingfisher.csv'
-rw-rw-r-- 1 root root 85959 Nov  3 18:43  parrot.csv
-rw-rw-r-- 1 root root 77909 Nov  3 18:37  peacock.csv
-rw-rw-r-- 1 root root 55118 Nov  3 19:05 'Red Headed Trogon.csv'
-rw-rw-r-

In [0]:
def download_url(url:str, dest:str, overwrite:bool=False, chunk_size=1024*1024, timeout=4):
    "Download `url` to `dest` unless it exists and not `overwrite`."
    if os.path.exists(dest) and not overwrite: return

    u = requests.get(url, stream=True, timeout=timeout)
    try: file_size = int(u.headers["Content-Length"])
    except: pass
        
    with open(dest, 'wb') as f:
        for chunk in u.iter_content(chunk_size=chunk_size):
            f.write(chunk)

def download_images(urls:str, dest:Path, max_pics:int=1000, max_workers:int=8, timeout=4,prefix=None,show_progress=True):
    "Download images listed in text file urls to path dest , at most max_pics"
    urls = open(urls).read().strip().split("\n")[:max_pics] # get all the urls in the list
    dest = Path(dest) # get the path of the folder 
    dest.mkdir(exist_ok=True) # create all the parent and child directory if neccessary. 

    if max_workers: # use the power of multiprocessing
        with ProcessPoolExecutor(max_workers=max_workers) as ex: # use a ProcessPoolExecutor for using pool of process to execute call asynchronously.
             # submit schedules the callable and return a Future object on completion. We iterate through all the urls to create Future objects for each url 
            futures = [ex.submit(download_url, url, dest/f"{i:08d}.jpg", timeout=timeout) for i,url in enumerate(urls)]
            if show_progress : 
                for f in tqdm(as_completed(futures), total=len(urls),desc=prefix): pass # show th progress bar

In [8]:
# get the path for all csv files
birdspath = Path("Birds")
birdspath = [ Path(child) for child in birdspath.iterdir()]
birdspath

[PosixPath('Birds/Oriental Dwarf Kingfisher.csv'),
 PosixPath('Birds/Red Headed Trogon.csv'),
 PosixPath('Birds/Asian Paradise Flycatcher.csv'),
 PosixPath('Birds/peacock.csv'),
 PosixPath('Birds/Sarus Crane.csv'),
 PosixPath('Birds/Himalayan Monal.csv'),
 PosixPath('Birds/parrot.csv'),
 PosixPath('Birds/Indian Pitta.csv'),
 PosixPath('Birds/Satyr Tragopan.csv'),
 PosixPath('Birds/Mrs. Gould’s Sunbird.csv')]

In [12]:
import re
from pprint import pprint 
all_birds_Name = [re.sub(r".csv|\.\s|\s*|\’" , r"", x.name) for x in birdspath] # Get all the birds name from the csv file
pprint(all_birds_Name)
folder_file = {x:y for x ,y in zip(all_birds_Name , birdspath)} # create a folder name the file name dictionary 
pprint(folder_file)

['OrientalDwarfKingfisher',
 'RedHeadedTrogon',
 'AsianParadiseFlycatcher',
 'peacock',
 'SarusCrane',
 'HimalayanMonal',
 'parrot',
 'IndianPitta',
 'SatyrTragopan',
 'MrsGould’sSunbird']
{'AsianParadiseFlycatcher': PosixPath('Birds/Asian Paradise Flycatcher.csv'),
 'HimalayanMonal': PosixPath('Birds/Himalayan Monal.csv'),
 'IndianPitta': PosixPath('Birds/Indian Pitta.csv'),
 'MrsGould’sSunbird': PosixPath('Birds/Mrs. Gould’s Sunbird.csv'),
 'OrientalDwarfKingfisher': PosixPath('Birds/Oriental Dwarf Kingfisher.csv'),
 'RedHeadedTrogon': PosixPath('Birds/Red Headed Trogon.csv'),
 'SarusCrane': PosixPath('Birds/Sarus Crane.csv'),
 'SatyrTragopan': PosixPath('Birds/Satyr Tragopan.csv'),
 'parrot': PosixPath('Birds/parrot.csv'),
 'peacock': PosixPath('Birds/peacock.csv')}


In [16]:
# download all the images 
for folder,file in folder_file.items():
    path = Path("train") # create a parent directory   
    dest = path/folder # Specific folder for each bird
    dest.mkdir(parents=True,exist_ok=True)
    download_images(file, dest,prefix=folder)

MrsGould’sSunbird: 100%|██████████| 550/550 [00:35<00:00, 15.47it/s]


In [25]:
length = lambda iterrator : len(tuple(iterrator))
birdsPerClassCount = { (path/folder).name : length((path/folder).iterdir()) for folder,file in folder_file.items() }
sorted(birdsPerClassCount.items() , key=lambda x :x[1])

[('HimalayanMonal', 510),
 ('SatyrTragopan', 531),
 ('OrientalDwarfKingfisher', 540),
 ('MrsGould’sSunbird', 544),
 ('RedHeadedTrogon', 578),
 ('SarusCrane', 605),
 ('AsianParadiseFlycatcher', 632),
 ('IndianPitta', 647),
 ('peacock', 803),
 ('parrot', 834)]

In [0]:
import PIL
def verify_image(file:Path, delete:bool ,dest:Path=None, n_channels:int=3, **kwargs):
    """Check if the image in `file` exists, can be opend and has `n_channels`. If `delete`, removes it if it fails.
        Result is stored in `dest` `img_format` and `kwargs` are passed to PIL.Image.save."""
    try:
        img = PIL.Image.open(file)
        assert isinstance(dest, Path), "You should provide `dest` Path to save resized image"
        if not file.is_file(): return
        if n_channels == 3: img = img.convert("RGB")
        img.save(file, **kwargs)
        img = np.array(img)
        img_channels = 1 if len(img.shape) == 2 else img.shape[2]
        assert img_channels == n_channels, f"Image {file} has {img_channels} instead of {n_channels}"
    except Exception as e:
        print(f'{e}')
        if delete: file.unlink()

def verify_images(path:Path, delete:bool=True, max_workers:int=4 , dest:Path=".",n_channels:int=3,**kwargs):
    """Check if the image in `path` exists, can be opened and has `n_channels`.
    If `n_channels` is 3 – it'll try to convert image to RGB. If `delete`, removes it if it fails.
    If `resume` – it will skip already existent images in `dest`.  If `max_size` is specifided,
    image is resized to the same ratio so that both sizes are less than `max_size`, using `interp`.
    Result is stored in `dest`, `ext` forces an extension type, `img_format` and `kwargs` are
    passed to PIL.Image.save. Use `max_workers` CPUs."""
    path = Path(path)
    dest = path/Path(dest)
    dest.mkdir(exist_ok=True)
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        files = [Path(file) for file in path.iterdir()]
        futures = [ex.submit(verify_image, file, delete=delete,dest=dest, n_channels=n_channels, **kwargs) for file in files]
    for f in tqdm(as_completed(futures), total=len(files) , desc=path.name): pass

In [27]:
for folder,file in folder_file.items():
    path = Path("train")
    verify_images(path/folder,delete=True,max_workers=8)

cannot identify image file '/content/train/OrientalDwarfKingfisher/00000542.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000394.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000539.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000204.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000023.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000383.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000528.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000007.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000364.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000129.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000439.jpg'
cannot identify image file '/content/train/OrientalDwarfKingfisher/00000124.jpg'
cannot identify image file '

OrientalDwarfKingfisher: 100%|██████████| 540/540 [00:00<00:00, 199535.21it/s]


cannot identify image file '/content/train/RedHeadedTrogon/00000152.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000065.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000221.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000283.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000348.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000177.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000342.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000364.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000298.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000400.jpg'


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


cannot identify image file '/content/train/RedHeadedTrogon/00000253.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000016.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000265.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000197.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000223.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000341.jpg'
cannot identify image file '/content/train/RedHeadedTrogon/00000076.jpg'


...

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:872: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


cannot identify image file '/content/train/SatyrTragopan/00000219.jpg'
cannot identify image file '/content/train/SatyrTragopan/00000191.jpg'
cannot identify image file '/content/train/SatyrTragopan/00000381.jpg'
cannot identify image file '/content/train/SatyrTragopan/00000285.jpg'
cannot identify image file '/content/train/SatyrTragopan/00000367.jpg'


SatyrTragopan: 100%|██████████| 531/531 [00:00<00:00, 190275.56it/s]


cannot identify image file '/content/train/MrsGould’sSunbird/00000306.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000114.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000065.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000494.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000217.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000423.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000474.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000344.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000412.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000113.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000153.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000508.jpg'
cannot identify image file '/content/train/MrsGould’sSunbird/00000406.jpg'
cannot identify image fil

MrsGould’sSunbird: 100%|██████████| 544/544 [00:00<00:00, 173263.07it/s]


In [28]:
birdsPerClassCountVerify = { (path/folder).name :len([*(path/folder).iterdir()]) for folder,file in folder_file.items() }
sorted(birdsPerClassCountVerify.items() , key=lambda x :x[1])

[('HimalayanMonal', 499),
 ('SatyrTragopan', 517),
 ('OrientalDwarfKingfisher', 519),
 ('MrsGould’sSunbird', 527),
 ('RedHeadedTrogon', 561),
 ('SarusCrane', 592),
 ('AsianParadiseFlycatcher', 604),
 ('IndianPitta', 629),
 ('parrot', 769),
 ('peacock', 776)]

In [29]:
# let's train our Images on a pretrained Resnet50 model
from keras.applications import ResNet50

image_input_shape = (256, 256, 3)

conv_base = ResNet50(weights='imagenet',
                  include_top=False,
                   classes=10,
                  input_shape=image_input_shape)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


In [30]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [37]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D(name='avg_pool'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 8, 8, 2048)        23587712  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [38]:
print(f'This is the number of trainable weights before freezing the conv base:{len(model.trainable_weights)}')
conv_base.trainable=False
print(f'This is the number of trainable weights after freezing the conv base:{len(model.trainable_weights)}')


This is the number of trainable weights before freezing the conv base:2
This is the number of trainable weights after freezing the conv base:2


In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [41]:
batch_size=32
# Use ImageDataGenerator with validation split of 20 % and  Image Augmentaion
datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=15, 
                                             brightness_range=(0.1,0.2) ,
                                             zoom_range=2, 
                                             horizontal_flip=True, 
                                             validation_split=0.2)

# use flow_from directory method in keras and give the parent directory name
train_generator = datagen.flow_from_directory(Path("train"),
                                              batch_size=batch_size,
                                              target_size=(256,256) , 
                                              subset="training")

validation_generator = datagen.flow_from_directory(Path("train"),
                                                   batch_size=batch_size,
                                                   target_size=(256,256) , 
                                                   subset="validation")


history = model.fit_generator(image_generator,
                              steps_per_epoch =train_generator.samples // batch_size, 
                              epochs=10, 
                              verbose=1,
                              workers=8,
                              validation_steps = validation_generator.samples // batch_size,
                              validation_data = validation_generator)


Found 4800 images belonging to 10 classes.
Found 1193 images belonging to 10 classes.
Epoch 1/10
150/150 [==============================] - 191s 1s/step - loss: 1.6792 - acc: 0.4319 - val_loss: 2.2606 - val_acc: 0.2610
Epoch 2/10
150/150 [==============================] - 177s 1s/step - loss: 1.2869 - acc: 0.5667 - val_loss: 2.0732 - val_acc: 0.2963
Epoch 3/10
150/150 [==============================] - 177s 1s/step - loss: 1.1946 - acc: 0.6043 - val_loss: 2.0628 - val_acc: 0.3342
Epoch 4/10
150/150 [==============================] - 170s 1s/step - loss: 1.1112 - acc: 0.6337 - val_loss: 2.4757 - val_acc: 0.2610
Epoch 5/10
150/150 [==============================] - 177s 1s/step - loss: 1.0747 - acc: 0.6363 - val_loss: 1.8439 - val_acc: 0.3824
Epoch 6/10
150/150 [==============================] - 175s 1s/step - loss: 1.0488 - acc: 0.6614 - val_loss: 2.2957 - val_acc: 0.3015
Epoch 7/10
150/150 [==============================] - 174s 1s/step - loss: 1.0578 - acc: 0.6579 - val_loss: 2.3381 -

# **THE** **END**
 
